In [ ]:
import pandas as pd
from google.colab import files

# Model


In [ ]:
!pip -q install pymorphy3 razdel nltk scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 31.3 MB/s eta 0:00:00


In [ ]:
import re
import ast
import json
import numpy as np
import pandas as pd

from razdel import tokenize
from pymorphy3 import MorphAnalyzer

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, classification_report
from joblib import dump


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [1]:
# Базовые RU стоп-слова
ru_stop = set(stopwords.words('russian'))
# Опционально
extra_stop = {
    "это","так","ещё","еще","просто","очень","свой","который","которые","которое"
}
ru_stop |= extra_stop

morph = MorphAnalyzer()
RE_URL  = re.compile(r"http\S+|www\.\S+", flags=re.IGNORECASE)
RE_MAIL = re.compile(r"\S+@\S+\.\S+")
RE_KEEP = re.compile(r"[a-zA-Zа-яА-Я0-9]+")

def normalize_basic(text: str) -> str:
    """Мягкая нормализация строки: прибиваем мусор, приводим к нижнему регистру."""
    if not isinstance(text, str):
        return ""
    t = text.replace("\xa0", " ").replace("\u200b", " ")
    t = RE_URL.sub(" ", t)
    t = RE_MAIL.sub(" ", t)
    t = t.replace("—", " ").replace("–", " ").replace("-", " ")
    t = re.sub(r"[^a-zA-Zа-яА-Я0-9 ]", " ", t)
    t = re.sub(r"\s+", " ", t).strip().lower()
    return t

def tokenize_lemma(text: str):
    """
    Токенайзер для TF-IDF:
    - нормализация,
    - razdel-токенизация,
    - фильтр по [a-zA-Zа-яА-Я0-9]+,
    - лемматизация pymorphy3,
    - удаление стоп-слов,
    - цифры оставляем как есть.
    """
    t = normalize_basic(text)
    out = []
    for tok in tokenize(t):
        w = tok.text
        if not RE_KEEP.fullmatch(w):
            continue
        if w.isdigit():
            out.append(w)
            continue
        lemma = morph.normal_forms(w)[0]
        if lemma in ru_stop:
            continue
        out.append(lemma)
    return out


NameError: name 'stopwords' is not defined

In [ ]:
tfidf = TfidfVectorizer(
    tokenizer=tokenize_lemma,
    preprocessor=None,
    token_pattern=None,     # обязательно отключаем дефолтный паттерн
    ngram_range=(1, 3),     # униграммы + биграммы
    min_df=5,
    max_df=0.8,
    sublinear_tf=True,
    max_features=200_000,   # подстрой под RAM/корпус
)


In [ ]:
dataset_train = pd.read_csv('dataset_train2.csv')
dataset_test = pd.read_csv('dataset_test.csv')

In [ ]:
dataset_train["llm_topics_parsed"] = dataset_train["llm_topics_parsed"].apply(ast.literal_eval)
dataset_test["llm_topics_parsed"]  = dataset_test["llm_topics_parsed"].apply(ast.literal_eval)

In [ ]:
# Бинаризация multi-label
mlb = MultiLabelBinarizer()
Y_train = mlb.fit_transform(dataset_train["llm_topics_parsed"])
Y_test  = mlb.transform(dataset_test["llm_topics_parsed"])

# TF-IDF матрицы
X_train = tfidf.fit_transform(dataset_train["clause"])
X_test  = tfidf.transform(dataset_test["clause"])

len_classes = len(mlb.classes_)
print("Classes:", len_classes, list(mlb.classes_))
print("Shapes:", X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)


Classes: 13 ['Автокредит', 'Вклады', 'Дебетовая карта', 'Денежные переводы', 'Дистанционное обслуживание', 'Другое', 'Ипотека', 'Кредитная карта', 'Мобильное приложение', 'Обслуживание', 'Потребительский кредит', 'Рефинансирование кредитов', 'Страхование']
Shapes: (37121, 12426) (37121, 13) (10679, 12426) (10679, 13)


In [ ]:
# 1) модель
clf = OneVsRestClassifier(
    LogisticRegression(
        max_iter=500,
        C=4.0,
        class_weight="balanced",
        solver="liblinear"
    )
)

# 2) обучение
clf.fit(X_train, Y_train)

# 3) вероятности на тесте
if hasattr(clf, "predict_proba"):
    Y_prob = clf.predict_proba(X_test)          # shape: (n_samples, n_classes)
else:
    scores = clf.decision_function(X_test)
    Y_prob = 1 / (1 + np.exp(-scores))

# 4) per-class пороги
DEFAULT_THR = 0.5
class_thresholds = {c: DEFAULT_THR for c in mlb.classes_}
class_thresholds["Страхование"] = 0.80  # пример: подняли порог

# вектор порогов по порядку mlb.classes_
thr_vec = np.array([class_thresholds[c] for c in mlb.classes_])    # shape: (n_classes,)

# 5) бинаризация
Y_pred = (Y_prob >= thr_vec).astype(int)   # broadcasting по столбцам


# 6) метрики
print("F1 micro:", f1_score(Y_test, Y_pred, average="micro"))
print("F1 macro:", f1_score(Y_test, Y_pred, average="macro"))
print("\nReport:\n", classification_report(Y_test, Y_pred, target_names=mlb.classes_, zero_division=0))

F1 micro: 0.6571872571872572
F1 macro: 0.6551532869955626

Report:
                             precision    recall  f1-score   support

                Автокредит       0.46      0.54      0.50       361
                    Вклады       0.72      0.77      0.75       662
           Дебетовая карта       0.64      0.79      0.71      1122
         Денежные переводы       0.65      0.74      0.69       676
Дистанционное обслуживание       0.55      0.68      0.61      1308
                    Другое       0.39      0.58      0.47       491
                   Ипотека       0.61      0.70      0.65       572
           Кредитная карта       0.59      0.74      0.65      1052
      Мобильное приложение       0.85      0.88      0.87       711
              Обслуживание       0.64      0.66      0.65      3164
    Потребительский кредит       0.50      0.73      0.59       864
 Рефинансирование кредитов       0.50      0.61      0.55       410
               Страхование       0.90      0.76

In [ ]:
import joblib
import os

# создаём папку, если её нет
save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

# сохраняем артефакты пайплайна
artifacts = {
    "tfidf": tfidf,
    "mlb": mlb,
    "clf": clf,
    "thresholds": class_thresholds,
}

joblib.dump(artifacts, os.path.join(save_dir, "model.pkl"))
print(f"✅ Модель сохранена в {save_dir}/model.pkl")

✅ Модель сохранена в models/model.pkl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/models
joblib.dump(artifacts, "/content/drive/MyDrive/models/model.pkl")

Mounted at /content/drive


['/content/drive/MyDrive/models/model.pkl']

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def tune_per_class_thresholds_micro(Y_prob, Y_true, init_thr=0.5, grid=None, max_iters=20, verbose=True):
    """
    Жадный координатный поиск индивидуальных порогов по классам
    для максимизации F1-micro. На каждой итерации перебираем порог
    отдельно для каждого класса, оставляя остальные фиксированными.
    """
    n_classes = Y_prob.shape[1]
    if grid is None:
        grid = np.linspace(0.05, 0.95, 19)

    thr_vec = np.full(n_classes, init_thr, dtype=float)

    def micro_f1_at(thr_v):
        Y_pred = (Y_prob >= thr_v).astype(int)  # broadcasting по столбцам
        return f1_score(Y_true, Y_pred, average="micro", zero_division=0)

    best_f1 = micro_f1_at(thr_vec)
    if verbose:
        print(f"[init] F1-micro={best_f1:.4f} (init_thr={init_thr})")

    improved = True
    it = 0
    while improved and it < max_iters:
        improved = False
        it += 1
        if verbose:
            print(f"\n[iter {it}]")

        for j in range(n_classes):
            base_thr_j = thr_vec[j]
            local_best_thr = base_thr_j
            local_best_f1 = best_f1

            # пробуем разные пороги для этого класса
            for t in grid:
                thr_vec[j] = t
                f1 = micro_f1_at(thr_vec)
                if f1 > local_best_f1 + 1e-8:
                    local_best_f1 = f1
                    local_best_thr = t

            # закрепляем лучший порог для класса j
            thr_vec[j] = local_best_thr
            if local_best_f1 > best_f1 + 1e-8:
                best_f1 = local_best_f1
                improved = True
                if verbose:
                    print(f"  class {j}: thr={local_best_thr:.2f}  ->  F1-micro={best_f1:.4f}")
            else:
                # откатываем, если улучшения нет
                thr_vec[j] = base_thr_j

    return thr_vec, best_f1

# пример использования:
thr_vec, f1p = tune_per_class_thresholds_micro(Y_prob, Y_test, init_thr=0.5, max_iters=3, verbose=True)
print("\nЛучшие per-class пороги (первые 10):", np.round(thr_vec[:10], 2))
print("F1-micro с per-class:", f1p)

# применяем:
Y_pred_perclass = (Y_prob >= thr_vec).astype(int)
print("F1-micro (проверка):", f1_score(Y_test, Y_pred_perclass, average="micro", zero_division=0))


[init] F1-micro=0.6540 (init_thr=0.5)

[iter 1]
  class 0: thr=0.75  ->  F1-micro=0.6562
  class 1: thr=0.60  ->  F1-micro=0.6565
  class 2: thr=0.55  ->  F1-micro=0.6565
  class 3: thr=0.55  ->  F1-micro=0.6565
  class 4: thr=0.65  ->  F1-micro=0.6573
  class 5: thr=0.70  ->  F1-micro=0.6586
  class 6: thr=0.65  ->  F1-micro=0.6608
  class 7: thr=0.65  ->  F1-micro=0.6623
  class 9: thr=0.35  ->  F1-micro=0.6636
  class 10: thr=0.65  ->  F1-micro=0.6654
  class 11: thr=0.85  ->  F1-micro=0.6688
  class 12: thr=0.70  ->  F1-micro=0.6692

[iter 2]
  class 2: thr=0.60  ->  F1-micro=0.6692

[iter 3]

Лучшие per-class пороги (первые 10): [0.75 0.6  0.6  0.55 0.65 0.7  0.65 0.65 0.5  0.35]
F1-micro с per-class: 0.6692291964833368
F1-micro (проверка): 0.6692291964833368


In [ ]:
def predict_topics(texts, class_thresholds=None, default_thr=0.5):
    X = tfidf.transform(pd.Series(texts))
    if hasattr(clf, "predict_proba"):
        prob = clf.predict_proba(X)
    else:
        score = clf.decision_function(X)
        prob = 1 / (1 + np.exp(-score))

    pred = []
    for row in prob:
        labels = []
        for c, p in zip(mlb.classes_, row):
            thr = class_thresholds.get(c, default_thr) if class_thresholds else default_thr
            if p >= thr:
                labels.append(c)
        pred.append(labels)

    return pred, prob

samples = [
""
           ]
thresholds = {"Страхование": 0.7}
labels, prob = predict_topics(samples, class_thresholds=thresholds, default_thr=0.5)
for s, l in zip(samples, labels):
    print("—", s, "\n  →", l)


—  
  → []


In [ ]:
# Восстановим исходные строки и метки
df_test = dataset_test.copy().reset_index(drop=True)

# Истинные метки
true_labels = [ [c for c, v in zip(mlb.classes_, row) if v == 1] for row in Y_test ]

# Предсказанные метки
pred_labels = [ [c for c, v in zip(mlb.classes_, row) if v == 1] for row in Y_pred ]

df_test["true_labels"] = true_labels
df_test["pred_labels"] = pred_labels

# Ошибки: либо не совпадает полностью, либо пусто
df_errors = df_test[df_test["true_labels"] != df_test["pred_labels"]]

print("Ошибок:", len(df_errors), "из", len(df_test))
df_errors[["clause", "true_labels", "pred_labels"]]

Ошибок: 6025 из 10679


,clause,true_labels,pred_labels
3,то связаться с сотрудниками банка можно будет ...,[Обслуживание],"[Дистанционное обслуживание, Обслуживание]"
6,проблему в приложении за столько времени (и на...,"[Мобильное приложение, Рефинансирование кредитов]",[Мобильное приложение]
7,"Почему Тиньков, Альфа банк и другие нормальные...",[Другое],[Кредитная карта]
8,тк карта была оформлена в рамках акции,[Дебетовая карта],"[Дебетовая карта, Кредитная карта]"
10,"Я ее подключил, вроде бы все ок",[],[Вклады]
...,...,...,...
10672,до которой опускался баланс вашего накопительн...,[],[Вклады]
10673,Сотрудница банка посоветовала повторно даже за...,[Рефинансирование кредитов],"[Обслуживание, Потребительский кредит]"
10676,"Оформил карту по ссылке, при которой при перво...",[Другое],"[Дебетовая карта, Другое, Кредитная карта]"
10677,карту на которую якобы придут кредитные деньги...,"[Дебетовая карта, Потребительский кредит]","[Дебетовая карта, Кредитная карта, Потребитель..."


In [ ]:
df_errors[:100]

,review_id,clause_id,clause,llm_topics,llm_sentiments,llm_topics_parsed,true_labels,pred_labels
3,4551,23,то связаться с сотрудниками банка можно будет ...,"[""Обслуживание""]","[""отрицательно""]",[Обслуживание],[Обслуживание],"[Дистанционное обслуживание, Обслуживание]"
6,2328,19,проблему в приложении за столько времени (и на...,"[""Мобильное приложение"", ""Рефинансирование кре...","[""отрицательно"", ""отрицательно""]","[Мобильное приложение, Рефинансирование кредитов]","[Мобильное приложение, Рефинансирование кредитов]",[Мобильное приложение]
7,2110,14,"Почему Тиньков, Альфа банк и другие нормальные...","[""Другое""]","[""нейтрально""]",[Другое],[Другое],[Кредитная карта]
8,211,3,тк карта была оформлена в рамках акции,"[""Дебетовая карта""]","[""нейтрально""]",[Дебетовая карта],[Дебетовая карта],"[Дебетовая карта, Кредитная карта]"
10,184,4,"Я ее подключил, вроде бы все ок",[],[],[],[],[Вклады]
...,...,...,...,...,...,...,...,...
179,2337,4,После положительного решения банка наведалась ...,"[""Обслуживание""]","[""нейтрально""]",[Обслуживание],[Обслуживание],[Автокредит]
183,776,10,из-за отказа в возврате средств и невозможност...,"[""Дистанционное обслуживание""]","[""отрицательно""]",[Дистанционное обслуживание],[Дистанционное обслуживание],[]
185,4781,4,позвонил на 16-й день) Не советую этот банки н...,[],[],[],[],[Обслуживание]
187,3828,4,только сказали что пол года назад закрытие так...,"[""Кредитная карта""]","[""отрицательно""]",[Кредитная карта],[Кредитная карта],[]
